# Projeto - Natural Language Processing (NLP)

**Integrantes**:
- Antonio Lucas Michelon de Almeida
- Pedro Nery Affonso dos Santos
- Rafael Gordon Paves

In [ ]:
import pandas as pd
import re
from dotenv import load_dotenv
import os
from google import genai
import json
from pydantic import BaseModel
import time
from tqdm import tqdm

load_dotenv()

In [2]:
# Captura pedido de música
query = input("O que você gostaria de ouvir hoje? ")

## Playlist gerada a partir de uma única requisição ao Gemini (LLM)

In [6]:
def monta_playlist(texto):
    """
    Função que realiza uma requisição de montagem de playlist diretamente para o Gemini.
    """
    
    prompt = f"""
    Você é um especialista em música e construção de playlists. 
    Você deve analisar o seguinte pedido de playlist e construir uma seleção de músicas que se encaixem no tema solicitado.
    Refira-se EXCLUSIVAMENTE ao pedido fornecido para criar a playlist.
    A playlist deve ser composta por 10 músicas, com o nome da música, do artista e possível featuring.
    A resposta deve ser uma lista em formato JSON, com os seguintes campos:
    - nome_musica: Nome da música
    - artista: Nome do artista
    - featuring: Nome do artista convidado (caso exista)

    Pedido do usuário: {texto}
    """
    
    class FormatoResposta(BaseModel):
        nome_musica: str
        artista: str
        featuring: str

    client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))
    resposta = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config={
            "response_mime_type": "application/json",
            'response_schema': list[FormatoResposta],
            'temperature': 1.0
            # 'max_output_tokens': 500,
        }
    )
    return resposta

# Resposta do Gemini
resposta = monta_playlist(query)
# Converte a resposta em um dicionário
resposta_dict = json.loads(resposta.text)
# Cria um DataFrame a partir da lista de dicionários
df = pd.DataFrame(resposta_dict)
# Renomeia as colunas
df.rename(columns={'nome_musica': 'Nome da Música', 'artista': 'Artista', 'featuring': 'Featuring'}, inplace=True)
df

,Nome da Música,Artista,Featuring
0,"Sugar, Sugar",The Archies,
1,Candy Shop,50 Cent,Olivia
2,I Want Candy,Bow Wow Wow,
3,Lollipop,Mika,
4,"Honey, Honey",ABBA,
5,Peaches,Justin Bieber,Daniel Caesar & Giveon
6,Strawberry Fields Forever,The Beatles,
7,Cherry Pie,Warrant,
8,Lemon Tree,Fools Garden,
9,Cake By The Ocean,DNCE,


## Playlist gerada a partir de interações entre agentes

## Playlist gerada a partir de BERT embeddings

In [2]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-large-cased')

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [3]:
unmasker("The most popular rock song is [MASK] from [MASK].")

[[{'score': 0.05607090890407562,
   'token': 1953,
   'token_str': 'Music',
   'sequence': '[CLS] The most popular rock song is Music from [MASK]. [SEP]'},
  {'score': 0.04651233181357384,
   'token': 6080,
   'token_str': 'Songs',
   'sequence': '[CLS] The most popular rock song is Songs from [MASK]. [SEP]'},
  {'score': 0.03482279181480408,
   'token': 2034,
   'token_str': 'originally',
   'sequence': '[CLS] The most popular rock song is originally from [MASK]. [SEP]'},
  {'score': 0.022510167211294174,
   'token': 2185,
   'token_str': 'Love',
   'sequence': '[CLS] The most popular rock song is Love from [MASK]. [SEP]'},
  {'score': 0.02219197154045105,
   'token': 1145,
   'token_str': 'also',
   'sequence': '[CLS] The most popular rock song is also from [MASK]. [SEP]'}],
 [{'score': 0.07174413651227951,
   'token': 26148,
   'token_str': 'U2',
   'sequence': '[CLS] The most popular rock song is [MASK] from U2. [SEP]'},
  {'score': 0.05013321712613106,
   'token': 10976,
   'token

In [ ]:
import pandas as pd
df = pd.read_csv('data/songs_filtrado.csv')
X = df.copy().drop(columns=['Tag', "Features", "Language"])
y = df['Tag']

from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import numpy as np
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
def get_embeddings(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state[0, 0, :]
    return cls_embedding

embeddings = []
for i in tqdm(range(len(X))):
    e = get_embeddings(X.iloc[i], model, tokenizer)
    print(f"{X.iloc[i]} :{e.detach().numpy()}")
    embeddings.append(e.detach().numpy())
embeddings = np.array(embeddings)